In [74]:
import pandas as pd
import numpy as np
from nameparser import HumanName
import re


In [75]:
# Input file is the Roster Download from College Website. It is a compilation of the available years.
df = pd.read_excel('YaleM2010.xlsx')

In [76]:
# Examine the columns, because there are a few different formats used by most colleges
df.columns

Index(['Field1_Text', 'Field1_Link', 'Field2', 'Field3', 'Field4', 'Field5',
       'Field6'],
      dtype='object')

In [77]:
# Rename columns with meaningful labels
df = df.rename(columns = {'Field1_Text':'Name','Field1_Link':'RosterLink','Field3':'ClassYear', 
                          'Field4':'Hometown','Field5':'HS','Field6':'RosterYear'})

In [78]:
# Note the 'Name' is in 3 possible columns, because the download of each roster by year 
# has a few different formats. Field4 is a duplicate name column, and will be ignored
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 7 columns):
Name          128 non-null object
RosterLink    128 non-null object
Field2        0 non-null float64
ClassYear     128 non-null object
Hometown      128 non-null object
HS            116 non-null object
RosterYear    128 non-null object
dtypes: float64(1), object(6)
memory usage: 7.1+ KB


In [79]:
# Preview the data
df.head()

,Name,RosterLink,Field2,ClassYear,Hometown,HS,RosterYear
0,Jordan Abergel,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Pacific Palisades, CA",Harvard Westlake,2009-10 Men's Tennis
1,Calvin Bennett,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Sandy, UT",The Waterford School,2009-10 Men's Tennis
2,Ryan Berman,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Armonk, NY",Bryam Hills,2009-10 Men's Tennis
3,Erik Blumenkranz,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Sophomore,"Portola Valley, Calif.",Sacred Heart Preparatory School,2009-10 Men's Tennis
4,Connor Dawson,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Senior,"St. Charles, IL",St. Francis,2009-10 Men's Tennis


In [80]:
# Combine and rename the 'Name' columns, to get a single column with no missing values
# The Name2 field was redundant so can be ignored
# For CornellM there is no 'Field4'
#df = df.assign(**{
#    'Name': df['Name1'].fillna(df['Name3'])})
#df = df.drop(['Name1', 'Name2', 'Name3'] ,axis=1)

In [81]:
# Preview again after consolidating Names columns
df.head()

,Name,RosterLink,Field2,ClassYear,Hometown,HS,RosterYear
0,Jordan Abergel,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Pacific Palisades, CA",Harvard Westlake,2009-10 Men's Tennis
1,Calvin Bennett,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Sandy, UT",The Waterford School,2009-10 Men's Tennis
2,Ryan Berman,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Junior,"Armonk, NY",Bryam Hills,2009-10 Men's Tennis
3,Erik Blumenkranz,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Sophomore,"Portola Valley, Calif.",Sacred Heart Preparatory School,2009-10 Men's Tennis
4,Connor Dawson,https://yalebulldogs.com/sports/mens-tennis/ro...,NaN,Senior,"St. Charles, IL",St. Francis,2009-10 Men's Tennis


In [82]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [83]:
#This name was on different rosters, once without an MI
#df['Name'][477] = 'James P. Shoffner'

In [84]:
# Some roster downloads have duplicates because of workarounds for the Octoparse roster looping. 
# Check the row count after dropping duplicates 
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 0 to 127
Data columns (total 7 columns):
Name          128 non-null object
RosterLink    128 non-null object
Field2        0 non-null float64
ClassYear     128 non-null object
Hometown      128 non-null object
HS            116 non-null object
RosterYear    128 non-null object
dtypes: float64(1), object(6)
memory usage: 8.0+ KB


In [85]:
# Reorder and sort the columns
df = df[['Name', 'RosterYear', 'ClassYear', 'Hometown', 'HS', 'RosterLink']]
df.sort_values(by=['Name', 'RosterYear'], inplace=True)
df.head()

,Name,RosterYear,ClassYear,Hometown,HS,RosterLink
38,Alan Sou,2017-18 Men's Tennis,First Year,Hong Kong,King George V,https://yalebulldogs.com/sports/mens-tennis/ro...
28,Alan Sou,2018-19 Men's Tennis,Sophomore,Hong Kong,King George V,https://yalebulldogs.com/sports/mens-tennis/ro...
19,Alan Sou,2019-20 Men's Tennis Roster,Junior,Hong Kong,King George V,https://yalebulldogs.com/sports/mens-tennis/ro...
82,Alex Hagermoser,2013-14 Men's Tennis,First Year,"Hinsdale, Ill.",Hinsdale Central,https://yalebulldogs.com/sports/mens-tennis/ro...
70,Alex Hagermoser,2014-15 Men's Tennis,Sophomore,"Hinsdale, Ill.",Hinsdale Central,https://yalebulldogs.com/sports/mens-tennis/ro...


In [86]:
# Describe is a quick way to see how many unique values are in each column 
df.describe()

,Name,RosterYear,ClassYear,Hometown,HS,RosterLink
count,128,128,128,128,116,128
unique,40,11,4,43,41,128
top,Daniel Hoffman,2010-11 Men's Tennis,Senior,"Atlanta, Ga.",St. Francis,https://yalebulldogs.com/sports/mens-tennis/ro...
freq,4,14,34,8,6,1


In [87]:
# Preview the groupby() stats, which will be used for aggreating to one row per unique name
df.groupby(['Name']).describe()

RosterYear                                           \
                      count unique                          top freq   
Name                                                                   
Alan Sou                  3      3         2017-18 Men's Tennis    1   
Alex Hagermoser           4      4         2014-15 Men's Tennis    1   
Andrew Heller             4      4         2017-18 Men's Tennis    1   
Arnav Dhingra             2      2         2018-19 Men's Tennis    1   
Calvin Bennett            2      2         2009-10 Men's Tennis    1   
Camille Kima              2      2         2018-19 Men's Tennis    1   
Charlie Adams             2      2         2016-17 Men's Tennis    1   
Cody Lin                  3      3         2017-18 Men's Tennis    1   
Connor Dawson             2      2         2009-10 Men's Tennis    1   
Daniel Faierman           4      4         2011-12 Men's Tennis    1   
Daniel Gale               1      1  2019-20 Men's Tennis Roster    1   
Daniel Hoffman            4      4         2009-10 Men's Tennis    1   
Dennis Wang               4      4         2017-18 Men's Tennis    1   
Dylan King                4      4         2017-18 Men's Tennis    1   
Erik Blumenkranz          3      3         2009-10 Men's Tennis    1   
Fedor Andrienko           4      4         2017-18 Men's Tennis    1   
James Ratchford           4      4         2014-15 Men's Tennis    1   
Jason Brown               4      4         2012-13 Men's Tennis    1   
Joel Samaha               3      3         2009-10 Men's Tennis    1   
John Huang                4      4         2009-10 Men's Tennis    1   
Jordan Abergel            2      2         2009-10 Men's Tennis    1   
Kyle Dawson               4      4         2011-12 Men's Tennis    1   
Louis Siegler             1      1  2019-20 Men's Tennis Roster    1   
Marc Powers               4      4         2009-10 Men's Tennis    1   
Martin Svenning           4      4         2012-13 Men's Tennis    1   
Matt Saiontz              4      4         2011-12 Men's Tennis    1   
Matt Schimmel             2      2         2009-10 Men's Tennis    1   
Michael Sun               2      2         2018-19 Men's Tennis    1   
Nathan Brown              4      4         2017-18 Men's Tennis    1   
Patrick Chase             4      4         2011-12 Men's Tennis    1   
Photos Photiades          4      4         2014-15 Men's Tennis    1   
Robert Shymansky          1      1  2019-20 Men's Tennis Roster    1   
Ryan Berman               2      2         2009-10 Men's Tennis    1   
Ryan Cheng                4      4         2017-18 Men's Tennis    1   
Stefan Doehler            4      4         2017-18 Men's Tennis    1   
Tommy Ratchford           4      4         2011-12 Men's Tennis    1   
Tyler Lu                  4      4         2014-15 Men's Tennis    1   
Zachary Dean              3      3         2011-12 Men's Tennis    1   
Zachary Krumholz          4      4         2011-12 Men's Tennis    1   
Ziqi Wang                 4      4         2017-18 Men's Tennis    1   

                 ClassYear                         Hometown         \
                     count unique         top freq    count unique   
Name                                                                 
Alan Sou                 3      3      Junior    1        3      1   
Alex Hagermoser          4      4      Senior    1        4      1   
Andrew Heller            4      4      Senior    1        4      2   
Arnav Dhingra            2      2  First Year    1        2      1   
Calvin Bennett           2      2      Senior    1        2      2   
Camille Kima             2      2      Senior    1        2      1   
Charlie Adams            2      2  First Year    1        2      1   
Cody Lin                 3      3      Junior    1        3      2   
Connor Dawson            2      1      Senior    2        2      1   
Daniel Faierman          4      4      Senior    1        4      1   
Daniel Gal

In [88]:
# Create copy of ClassYear in order to aggregate count and keep original unique aggregation when grouping
df['YearsPlayed'] = df['ClassYear']

In [89]:
# Create 'Year' by 1) converting RosterYear to a list
# 2) Loop through the list to use RegEx split function
# 3) Use list comprehension to extract first column of each split row
# 4) Convert string to integer and add 1 to calculate value from first year before dash in yyyy-yy
rosterlist = df['RosterYear'].tolist()
Year=[]
for i in rosterlist:
    Split=int(re.split(r'\D+', i)[0])+1
    Year.append(Split)
df['Year'] = Year

In [90]:
# Use groupby() with aggregation to consolidate columns and condense to one row per Name
dfgroup = df.groupby('Name').aggregate({'Name':'first', 'Year':'max', 'YearsPlayed':'count',\
'RosterYear':'unique', 'ClassYear':'unique', 'Hometown':'first', 'HS':'first', 'RosterLink':'last'})

In [91]:
# Add a column for College, Criteris, and Gender
dfgroup['College'] = 'Yale University'
dfgroup['Criteria'] = ' Yale'
dfgroup['Gender'] = 'M'

In [92]:
dfgroup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, Alan Sou to Ziqi Wang
Data columns (total 11 columns):
Name           40 non-null object
Year           40 non-null int64
YearsPlayed    40 non-null int64
RosterYear     40 non-null object
ClassYear      40 non-null object
Hometown       40 non-null object
HS             37 non-null object
RosterLink     40 non-null object
College        40 non-null object
Criteria       40 non-null object
Gender         40 non-null object
dtypes: int64(2), object(9)
memory usage: 3.8+ KB


In [93]:
dfgroup.sort_values('Year')

,Name,Year,YearsPlayed,RosterYear,ClassYear,Hometown,HS,RosterLink,College,Criteria,Gender
Name,,,,,,,,,,,
Connor Dawson,Connor Dawson,2011,2,"[2009-10 Men's Tennis, 2010-11 Men's Tennis]",[Senior],"St. Charles, IL",St. Francis,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Ryan Berman,Ryan Berman,2011,2,"[2009-10 Men's Tennis, 2010-11 Men's Tennis]","[Junior, Senior]","Armonk, NY",Bryam Hills,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Matt Schimmel,Matt Schimmel,2011,2,"[2009-10 Men's Tennis, 2010-11 Men's Tennis]",[Senior],"Jacksonville, FL",Bolles School,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Calvin Bennett,Calvin Bennett,2011,2,"[2009-10 Men's Tennis, 2010-11 Men's Tennis]","[Junior, Senior]","Sandy, UT",The Waterford School,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Jordan Abergel,Jordan Abergel,2011,2,"[2009-10 Men's Tennis, 2010-11 Men's Tennis]","[Junior, Senior]","Pacific Palisades, CA",Harvard Westlake,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Erik Blumenkranz,Erik Blumenkranz,2012,3,"[2009-10 Men's Tennis, 2010-11 Men's Tennis, 2...","[Sophomore, Junior, Senior]","Portola Valley, Calif.",Sacred Heart Preparatory School,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Joel Samaha,Joel Samaha,2012,3,"[2009-10 Men's Tennis, 2010-11 Men's Tennis, 2...","[Sophomore, Junior, Senior]","Tampa, Fla.",H.B. Plant High School,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Zachary Dean,Zachary Dean,2013,3,"[2010-11 Men's Tennis, 2011-12 Men's Tennis, 2...","[Sophomore, Junior, Senior]","Commack, N.Y.",Commack,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M
Daniel Hoffman,Daniel Hoffman,2013,4,"[2009-10 Men's Tennis, 2010-11 Men's Tennis, 2...","[First Year, Sophomore, Junior, Senior]","Atherton, Calif.",NaN,https://yalebulldogs.com/sports/mens-tennis/ro...,Yale University,Yale,M


In [94]:
# Use a for loop with nameparser utility HumanName to parse names into First, Middle, Last
namelist = dfgroup['Name'].tolist()
First=[]
Last=[]
Middle=[]
for name in namelist:
    parsename = HumanName(name)
    First.append(parsename.first)
    Last.append(parsename.last)
    Middle.append(parsename.middle)

In [95]:
# Display the elements of parsename.HumanName for last row. title, suffix, nickname are not applicable 
parsename

<HumanName : [
	title: '' 
	first: 'Ziqi' 
	middle: '' 
	last: 'Wang' 
	suffix: ''
	nickname: ''
]>

In [96]:
# Use RegEx to identify Middle values with a period vs. longer names with no period
# Note RegEx syntax "?<!\w\" means don't split the character e.g. keep the period
MidSplit=[]
for i in Middle:
    MidSplit.append(re.split(r'\.*\W+', i)[0])


In [97]:
MiddleName=[]
for i in MidSplit:
    if len(i) > 1:
        MiddleName.append(" "+i)
    else:
        MiddleName.append("")


In [98]:
dfgroup['First'] = First
dfgroup['MidInit'] = Middle
dfgroup['MidName'] = MiddleName
dfgroup['Last'] = Last
dfgroup['FirstLast'] = dfgroup['First'].map(str)+dfgroup['MidName']+' '+dfgroup['Last'].map(str)

In [99]:
dfgroup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, Alan Sou to Ziqi Wang
Data columns (total 16 columns):
Name           40 non-null object
Year           40 non-null int64
YearsPlayed    40 non-null int64
RosterYear     40 non-null object
ClassYear      40 non-null object
Hometown       40 non-null object
HS             37 non-null object
RosterLink     40 non-null object
College        40 non-null object
Criteria       40 non-null object
Gender         40 non-null object
First          40 non-null object
MidInit        40 non-null object
MidName        40 non-null object
Last           40 non-null object
FirstLast      40 non-null object
dtypes: int64(2), object(14)
memory usage: 5.3+ KB


In [100]:
# Unique name count is 229, with Roster year count 39. There is a gap missing 1966-67 through 2000-2001, or 34 missing years. 
# 227 LinkedIn URL values were found, but many are likely to be non-matching. 
# The earliest roster year is 1946-47, and there will be dropff of found URL's at some point
# Collapse to unique names, and create boolean for roster years (39 columns)

In [101]:
dfpdf = pd.read_csv('YaleMpdf.csv')

In [102]:
dfpdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 7 columns):
FirstLast      384 non-null object
Last           384 non-null object
First          384 non-null object
MidInit        271 non-null object
MidName        35 non-null object
Year           384 non-null int64
YearsPlayed    384 non-null int64
dtypes: int64(2), object(5)
memory usage: 21.1+ KB


In [53]:
#dfpdf['FirstLastlower'] = dfpdf['FirstLast'].str.lower()

In [103]:
#dfname = dfgroup[['FirstLast']]
#dfname['namelower'] = dfgroup['FirstLast'].str.lower()

In [104]:
#df2 = pd.merge(dfname, dfpdf,
#                     left_on='namelower', right_on='FirstLastlower', how='outer',suffixes=('_R','_L'))
#df2.rename(columns={'FirstLast_L': 'FirstLast'}, inplace=True)

In [105]:
#df2 = df2[(df2['namelower'] != df2['FirstLastlower'])]

In [106]:
#df2.dropna(subset = ['FirstLast'], inplace=True)

In [107]:
#df2.drop(columns=['FirstLast_R', 'namelower', 'FirstLastlower'], inplace=True)

In [108]:
#df2 = df2[df2.FirstLast != 'Tom RATCHFORD']

In [109]:
#df2 = df2[df2.FirstLast != 'Zach DEAN']

In [110]:
#df2.info()

In [111]:
#df2

In [112]:
dfappend = dfgroup.append(dfpdf)

In [113]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [114]:
#dfappend.info()

In [115]:
dfappend.reset_index(drop=True)
dfappend.set_index(['FirstLast'], inplace=True, drop=False)

In [116]:
# Update College, Critera, and Gender for appended names
dfappend['College'] = 'Yale University'
dfappend['Criteria'] = ' Yale'
dfappend['Gender'] = 'M'

In [117]:
cols_to_order = ['Criteria','FirstLast', 'Year', 'YearsPlayed', 'RosterLink', 'Hometown', 'HS']
new_columns = cols_to_order + (dfappend.columns.drop(cols_to_order).tolist())
dfappend = dfappend[new_columns]

In [118]:
# Write the new format to a csv file
#dfappend.to_csv('CornellM_RosterGroup.csv', index=True)

In [119]:
dfappend.sort_values(by=['Year'], inplace=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
dfappend

,Criteria,FirstLast,Year,YearsPlayed,RosterLink,Hometown,HS,ClassYear,College,First,Gender,Last,MidInit,MidName,Name,RosterYear
FirstLast,,,,,,,,,,,,,,,,
Victor Kiam,Yale,Victor Kiam,1947,1,NaN,NaN,NaN,NaN,Yale University,Victor,M,Kiam,K.,NaN,NaN,NaN
Charles Ritchie,Yale,Charles Ritchie,1947,2,NaN,NaN,NaN,NaN,Yale University,Charles,M,Ritchie,L.,Jr,NaN,NaN
James Boyle,Yale,James Boyle,1947,1,NaN,NaN,NaN,NaN,Yale University,James,M,Boyle,T.,Jr,NaN,NaN
Henry Ewald,Yale,Henry Ewald,1947,1,NaN,NaN,NaN,NaN,Yale University,Henry,M,Ewald,T.,Jr,NaN,NaN
C. Wesley Kittleman,Yale,C. Wesley Kittleman,1947,1,NaN,NaN,NaN,NaN,Yale University,C. Wesley,M,Kittleman,NaN,Jr,NaN,NaN
Robert Sutton,Yale,Robert Sutton,1947,1,NaN,NaN,NaN,NaN,Yale University,Robert,M,Sutton,J.,NaN,NaN,NaN
Charles Clark,Yale,Charles Clark,1947,2,NaN,NaN,NaN,NaN,Yale University,Charles,M,Clark,H.,NaN,NaN,NaN
Miles Suchin,Yale,Miles Suchin,1947,2,NaN,NaN,NaN,NaN,Yale University,Miles,M,Suchin,B.,NaN,NaN,NaN
George Bell,Yale,George Bell,1947,2,NaN,NaN,NaN,NaN,Yale University,George,M,Bell,deB.,NaN,NaN,NaN


In [120]:
# Write the new format to a csv file
dfappend.to_csv('YaleM2_group.csv', index=True)

In [121]:
dfappend['FirstLast'].to_csv('NamelistYaleM2.csv', index=False, header=True)
dfappend['Criteria'].to_csv('CriterialistYaleM2.csv', index=False, header=True)

In [122]:
dfappend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, Victor Kiam to Alan Sou
Data columns (total 16 columns):
Criteria       424 non-null object
FirstLast      424 non-null object
Year           424 non-null int64
YearsPlayed    424 non-null int64
RosterLink     40 non-null object
Hometown       40 non-null object
HS             37 non-null object
ClassYear      40 non-null object
College        424 non-null object
First          424 non-null object
Gender         424 non-null object
Last           424 non-null object
MidInit        311 non-null object
MidName        75 non-null object
Name           40 non-null object
RosterYear     40 non-null object
dtypes: int64(2), object(14)
memory usage: 56.3+ KB
